<a href="https://colab.research.google.com/github/SpoilStick/ML-Projects/blob/main/ML%20Assignment%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Part 1: Data Exploration

> Indented block



---

In [ ]:
diabetes = datasets.load_diabetes()

In [ ]:
print(diabetes.DESCR)

In [ ]:
data = pd.DataFrame(diabetes.data)
features = diabetes.feature_names
data.columns = features

target = diabetes.target

data['target'] = target

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
train, test =  train_test_split(data, test_size=0.2, random_state=42)
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
print("Train:", len(train), "Test:", len(test))

In [ ]:
corr_matrix = data.corr()
corr_matrix["target"].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,10))
ax= sns.heatmap(corr_matrix, annot=True)
plt.show()

sns.pairplot(data[["bmi","s5","bp"]])

In [ ]:
y_train, y_test = train['target'], test['target']
X_train, X_test = train.drop(columns='target'), test.drop(columns='target')

# Part 2: Model Training

---

In [ ]:
def print_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Std:", scores.std())

**Linear Regression**

In [ ]:
lin_reg = LinearRegression().fit(X_train ,y_train)
lin_predictions = lin_reg.predict(X_train)

RMSE = mean_squared_error(y_train, lin_predictions, squared=False)
print("RMSE:", RMSE)

lin_cv_scores = cross_val_score(lin_reg, X_train, y_train)
lin_rmse_scores = np.sqrt(lin_cv_scores)
print_scores(lin_rmse_scores)

**Random Forest Regressor**

In [ ]:
forest_reg = RandomForestRegressor(random_state=42).fit(X_train ,y_train)
forest_predictions = forest_reg.predict(X_train)

RMSE = mean_squared_error(y_train, forest_predictions, squared=False)
print("RMSE:", RMSE)

forest_cv_scores = cross_val_score(forest_reg, X_train, y_train)
forest_rmse_scores = np.sqrt(forest_cv_scores)
print_scores(forest_rmse_scores)

# Part 3: Model Tuning

---



In [ ]:
forest_reg.get_params()

In [ ]:
param_grid = [
              {'n_estimators':  [3, 10, 30, 100], 'max_features': [2, 4, 6, 8]},
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2,3,4]},
]

grid_search = GridSearchCV(forest_reg, param_grid, cv=5).fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)

In [ ]:
pd.DataFrame(cvres)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, features), reverse=True)

In [ ]:
RMSE = mean_squared_error(y_train, forest_predictions, squared=False)
print("RMSE:", RMSE)

**Correlation Matrix:** bmi s5 bp s4 s6 s1 age s2 sex s3

**Feature Importances**: bmi s5 bp s3 s2 s4 s6 s1 age sex

In [ ]:
best_param_model = grid_search.best_estimator_
best_param_predictions = best_param_model.predict(X_train)

best_param_RMSE = mean_squared_error(y_train, best_param_predictions, squared=False)
print(best_param_RMSE)

#**Part 4: Model Evaluation**


*   Assignment 1 Linear Regression RMSE: 59.21
*   Assignment 2 Linear Regression RMSE: 53.56
*   Assignment 2 Random Forest Regressor RMSE: 21.46
*   Assignment 2 RFR with grid search parameters RMSE:  21.31